In [1]:
import sqlalchemy as sqla
import getpass
import pymssql

user = 'tsquare_app_user'
password = getpass.getpass()
server = 'tsquare-public.cwgloob09acb.us-east-1.rds.amazonaws.com'
engine = sqla.create_engine('mssql+pymssql://' + user + ':' + password + '@' + server)
connection = engine.connect()
print(connection)

········


In [2]:
import pandas as pd

query = """
SELECT 
	EncounterNumber,
    V.[Name] AS VisitType,
    SUM([Time]) AS ETime
FROM 
	TSQUARE.Facts.EncounterPatientFlow PF
INNER JOIN
    TSQUARE.Dimensions.VisitType V
ON
    PF.VisitType_Id = V.Id
WHERE
	YEAR(AppointmentTime) >= 2015
GROUP BY 
    EncounterNumber,
    V.[Name]
HAVING 
    SUM([Time]) > 0
;
"""
    
df = pd.read_sql_query(sql = query , con=connection)

In [3]:
df_summary = df.groupby('ETime', as_index=False).count()
df_summary.drop('VisitType', axis=1, inplace=True)
df_summary.columns = ['ETime', 'NumberOfEncounters']
df_summary['VisitType']='(All)'

df_summary_with_type = df.groupby(['ETime', 'VisitType'], as_index=False).count()
df_summary_with_type.columns = ['ETime', 'VisitType', 'NumberOfEncounters']

df = pd.concat([df_summary, df_summary_with_type])

In [4]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure

output_notebook()

Loading BokehJS ...

In [5]:
from bokeh.models import HoverTool, ColumnDataSource


hover = HoverTool(
        tooltips=[
            ("(ETime,NumberOfEncounters)", "($x, $top)"),
        ]
    )

visit_type = '(All)'

source = ColumnDataSource({'x': df[df.VisitType == visit_type].ETime,
                           'top': df[df.VisitType == visit_type].NumberOfEncounters
})

In [6]:
from ipywidgets import interact
from bokeh.io import push_notebook
from bokeh.models.widgets import Select
from bokeh.models.annotations import Title
from bokeh.models.ranges import Range1d

plot = figure(plot_width=900, 
              plot_height=800, 
              x_range=(0,500),
              tools=[hover,'pan', 'wheel_zoom', 'box_select'],
              title='Distribution of Encounters by LOS (2015-Present)'
            )

plot.title.text_font_size = '18pt'
plot.xaxis.axis_label = 'Encounter time (min.)'
plot.xaxis.axis_label_text_font_size= '16pt'
plot.yaxis.axis_label = 'Number of Encounters'
plot.yaxis.axis_label_text_font_size= '16pt'

hist = plot.vbar('x',
                 top='top',
                 source=source,
                 width=1.0,
                 color='#222222',
                 alpha=0.7)

p = ''

def update(visit_type):
    hist.data_source.data['top'] = df[df.VisitType == visit_type].NumberOfEncounters
    hist.data_source.data['x'] = df[df.VisitType == visit_type].ETime
    push_notebook(handle=p)
    
p = show(plot, notebook_handle=True)

In [7]:
interact(update, visit_type=list(df.sort_values('NumberOfEncounters', ascending=False).VisitType.unique()))

A Jupyter Widget

<function __main__.update>

In [8]:
query2 = """
SELECT 
	EncounterNumber,
    V.[Name] AS VisitType,
	RTRIM(LTRIM(STR(ROUND(PAT.Age - 1, -1)))) + '-' + RTRIM(LTRIM(STR(ROUND(PAT.Age + 9, -1)))) AS AgeBin,
    ROUND(SUM([Time])-3, -1) + 5 AS ETime
FROM 
	TSQUARE.Facts.EncounterPatientFlow PF
INNER JOIN
    TSQUARE.Dimensions.VisitType V
ON
    PF.VisitType_Id = V.Id
INNER JOIN 
	TSQUARE.Dimensions.Patient PAT
ON
	PF.Patient_Id = PAT.Id
WHERE
	YEAR(AppointmentTime) >= 2015
GROUP BY 
    EncounterNumber,
    V.[Name],
	RTRIM(LTRIM(STR(ROUND(PAT.Age - 1, -1)))) + '-' + RTRIM(LTRIM(STR(ROUND(PAT.Age + 9, -1))))
HAVING 
    SUM([Time]) > 0
"""

df2 = pd.read_sql_query(sql = query2 , con=connection)

In [9]:
df2_summary = df2.groupby(['ETime', 'AgeBin'], as_index=False).count()
df2_summary.drop('VisitType', axis=1, inplace=True)
df2_summary.columns = ['ETime', 'AgeBin', 'NumberOfEncounters']
df2_summary['VisitType']='(All)'

df2_summary_with_type = df2.groupby(['ETime', 'VisitType', 'AgeBin'], as_index=False).count()
df2_summary_with_type.columns = ['ETime', 'VisitType', 'AgeBin', 'NumberOfEncounters']

df_age = pd.concat([df2_summary, df2_summary_with_type], ignore_index=True)
df_age

,AgeBin,ETime,NumberOfEncounters,VisitType
0,0-10,5.0,25,(All)
1,10-20,5.0,64,(All)
2,100-110,5.0,5,(All)
3,20-30,5.0,59,(All)
4,30-40,5.0,57,(All)
5,40-50,5.0,101,(All)
6,50-60,5.0,204,(All)
7,60-70,5.0,374,(All)
8,70-80,5.0,637,(All)
9,80-90,5.0,386,(All)


In [10]:
hover = HoverTool(
        tooltips=[
            ("(ETime,NumberOfEncounters)", "($x, $top)"),
        ]
    )

visit_type2 = '(All)'
age_bin = '50-60'

source = ColumnDataSource({'x': df_age[(df_age.VisitType == visit_type2) & (df_age.AgeBin == age_bin)].ETime,
                           'top': df_age[(df_age.VisitType == visit_type2) & (df_age.AgeBin == age_bin)].NumberOfEncounters
})

plot2 = figure(plot_width=900, 
              plot_height=800, 
              x_range=(0,500),
              tools=[hover,'pan', 'wheel_zoom', 'box_select'],
              title='Distribution of Encounters for Different Age Groups by LOS (2015-Present)'
            )

plot2.title.text_font_size = '18pt'
plot2.xaxis.axis_label = 'Encounter time (min.)'
plot2.xaxis.axis_label_text_font_size= '16pt'
plot2.yaxis.axis_label = 'Number of Encounters'
plot2.yaxis.axis_label_text_font_size= '16pt'

hist2 = plot2.vbar('x',
                 top='top',
                 source=source,
                 width=1.0,
                 color='#37557C',
                 alpha=0.7)

p2 = ''

def update_age(visit_type, age_bin):
    hist2.data_source.data['top'] = df_age[(df_age.VisitType == visit_type) & (df_age.AgeBin == age_bin)].NumberOfEncounters
    hist2.data_source.data['x'] = df_age[(df_age.VisitType == visit_type) & (df_age.AgeBin == age_bin)].ETime
    push_notebook(handle=p2)

p2 = show(plot2, notebook_handle=True)

In [11]:
interact(update_age, 
         visit_type=list(df_age.sort_values('NumberOfEncounters', ascending=False).VisitType.unique()),
         age_bin=list(df_age.AgeBin.unique()))

A Jupyter Widget

<function __main__.update_age>

In [12]:
df2.index

RangeIndex(start=0, stop=169838, step=1)

In [13]:
help(figure.xaxis.axis_label_text_font_size)

AttributeError: 'function' object has no attribute 'xaxis'